<a href="https://colab.research.google.com/github/AriannaBi/SimpleFCNN/blob/main/SimpleFCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import csv
import torch

Function to read the name of the files

In [58]:
def type_file(name):
    name_file = name.split('/')[3][:3]
    # print(name_file)
    if name_file == 'EDA':
        return 0, 0
    elif name_file == 'BVP':
        return '250ms', 1
    elif name_file == 'ACC':
        return '250ms', 2
    elif name_file == 'TEM':
        return 0, 5
    return -1

In [73]:
def read_data_locally(files):
    leng = 0
    for name_file in files:
        table = pd.read_parquet(name_file, engine='pyarrow')
        df = pd.DataFrame(table)
        timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
        df['timestamp'] = timestamp_col
        arr_type = type_file(name_file)
        if arr_type == -1:
            print("Error in the file name")
            return -1
        if arr_type[0] != 0:
            # print("resample stage")
            df = df.resample(arr_type[0], on='timestamp').mean()
            df = df.reset_index()
        leng += df.shape[0]//2400 + int(df.shape[0] % 2400 > 0)
    tensor = np.zeros((6, leng, 2400),dtype=np.float64)



     # NOW READ ALL THE FILES AND FILL THE TENSOR IN THE CORRESPONDING PLACE
    arr_idx = [0,0,0,0,0,0]
    for name_file in files:
      # array of indices of row to fill
      # EDA, BVP, ACCX, ACCY, ACCZ, TEMP
      
      # DataFrame
      table = pd.read_parquet(name_file, engine='pyarrow')
      # creating DataFrame
      df = pd.DataFrame(table)
      # converting timestamp
      timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
      df['timestamp'] = timestamp_col
      # get if file it's among EDA, BVP, ACC, ST, otherwise return error
      arr_type = type_file(name_file)
      if arr_type == -1:
          print("Error in the file name")
          return -1

      # RESAMPLE the tensor
      # 0 means it's already 4Hz, otherwise resample with '250ms'
      if arr_type[0] != 0:
          # print("resample stage")
          df = df.resample(arr_type[0], on='timestamp').mean()
          df = df.reset_index()



      # FILL the tensor
      # number_rows = df.shape[0]//2400 + int(df.shape[0] % 2400 > 0)
      number_rows = df.shape[0]//2400
      print(number_rows, df.shape[0])
      # if not ACC, fill the #st matrix
      if arr_type[1] != 2:
          # print("fill tensor stage not ACC")
          position_element_in_column = 0
          row_to_fill = arr_idx[arr_type[1]]

          for j in range(row_to_fill, number_rows):
            # everytime it enter the loop add one row
              arr_idx[arr_type[1]] += 1
              for k in range(2400):
                  tensor[arr_type[1]][j][k] = df['value'][position_element_in_column]
                  position_element_in_column += 1
      # otherwise fill ACC_X, ACC_Y, ACC_Z
      elif arr_type[1] == 2:
          # print("fill tensor stage of ACC")
          list_axis_ACC = ['X', 'Y', 'Z']
          for idx, axis in enumerate(list_axis_ACC):
              index = 2 + idx
              row_to_fill = arr_idx[index]
              position_element_in_column = 0
              for j in range(row_to_fill, number_rows):
                  for k in range(2400):
                      tensor[index][j][k] = df[axis][position_element_in_column]
                      position_element_in_column += 1
          
      
    print(tensor)
    # print(arr_idx)

    # for elem in tensor[1]:
    #     print(elem)




      # with open('modified.csv', 'w') as csvfile:
      #       fieldnames = ['Date', 'Value']
      #       writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      #       writer.writeheader()
      #       for i in range(0, df.shape[0]):
      #           writer.writerow({'Date': df['timestamp'][i], 'Value': df['value'][i]})


      
      # for i in range(0, len(tensor[1])):
      #   for j in range(0, len(tensor[1][i])):
      #     print(tensor[1][i][j])

      


          

list_files = ['./drive/MyDrive/ACC_22.parquet', './drive/MyDrive/BVP_41.parquet', './drive/MyDrive/BVP_26.parquet', './drive/MyDrive/TEMP_25.parquet']
# './drive/MyDrive/ACC_07.parquet', ]
read_data_locally(list_files)


126 304393
52 124800
83 199372
53 129024
[[[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0

prendo il tensor, leggo un file e faccio reshsape di tutto in base a quella dimensione.
reshape per ogni matrix tengo un indice che mi dice la riga da cui devo iniziare per fare il fill.

oppure leggo un file e costruisco una list, poi converto la list in tensor or np.array

In [ ]:
# create 6 matrices with rows and cols ( create TENSOR)
        # i - z axis            6
        # j - y axis (rows)     df.shape[0]/2400 (how many windows)
        # k - x axis (cols)     2400
        # number_rows = int(df.shape[0] // 2400)
        # tensor = torch.zeros((6, number_rows, 2400), dtype=torch.float64)
        # print(tensor)

        # # FILL the tensor
        # # if not ACC, fill the #st matrix
        # if arr_type[1] != 2:
        #     print("fill tensor stage not ACC")
        #     position_element_in_column = 0
        #     for j in range(number_rows):
        #         for k in range(2400):
        #             tensor[arr_type[1]][j][k] = df['value'][position_element_in_column]
        #             position_element_in_column +=  1
        #     print(tensor)

        # # otherwise fill ACC_X, ACC_Y, ACC_Z
        # elif arr_type[1] == 2:
        #     print("fill tensor stage of ACC")
        #     list_axis_ACC = ['X', 'Y', 'Z']
        #     for idx, axis in enumerate(list_axis_ACC):
        #         position_element_in_column = 0
        #         for j in range(number_rows):
        #             for k in range(2400):
        #                 tensor[idx+2][j][k] = df[axis][position_element_in_column]
                        # position_element_in_column += 1
            # print(tensor)

In [ ]:
# x = np.array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.,10.,11.])
# x = x.reshape(2,2,3)
x = np.zeros((3,2,2))
# calc lenght = 9 elem
lista = [11.,22.]
# len = list.lenght/2 + (list.lenght % 2 >0)
# len
x[0][0][0] = 1.
x[0][0][1] = 2.
x[0][1][0] = 3.
x[0][1][1] = 4.

# x[0] = np.vstack((x[0],lista))
x
# # print(x , '\n\n\n')
# y = np.array([111,222,333,444,555,666,777,888,999,100,111,122])
# y = y.reshape(2,2,3)
# # print(y , '\n\n\n')
# z = np.concatenate((x[0], y[0]), axis = 1)
# # z
# b = [[[1,2,], [3,4]], [[5,6]]]
# a = [[7,8]]
# b = b[0] + a
# # x[0] = z



# a = np.array(b)
# a

ValueError: ignored